In [11]:
import os
%matplotlib inline
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
from patsy import dmatrices
data_path = '/Users/kylieleblancKylie/domino/dofis/data/'
table_path = '/Users/kylieleblancKylie/domino/dofis/results/descriptive/'
data = pd.read_csv(os.path.join(data_path, 'clean', 'cits_match.csv'),
                  sep=",", low_memory=False)

# No Matching

In [66]:
y, X = dmatrices('avescores ~  treat + post + doi_year_centered + yearpost + treatyear + treatpost + treatpostyear + treatpostyear2',
                 data=data, return_type='dataframe', NA_action = "drop")
cluster = data[pd.notnull(data.avescores)].distname
mod = smf.OLS(y, X)    # Describe model
res = mod.fit(cov_type = 'cluster',  cov_kwds={'groups': pd.DataFrame(cluster)})       # Fit model
print(res.summary())   # Summarize model

                            OLS Regression Results                            
Dep. Variable:              avescores   R-squared:                       0.077
Model:                            OLS   Adj. R-squared:                  0.076
Method:                 Least Squares   F-statistic:                     69.07
Date:                Thu, 14 Mar 2019   Prob (F-statistic):           5.83e-87
Time:                        21:33:14   Log-Likelihood:                -6506.5
No. Observations:                6302   AIC:                         1.303e+04
Df Residuals:                    6293   BIC:                         1.309e+04
Df Model:                           8                                         
Covariance Type:              cluster                                         
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept             0.1348      0.05

# Matching


In [67]:
y, X = dmatrices('avescores ~  treat + post + doi_year_centered + yearpost + treatyear + treatpost + treatpostyear + treatpostyear2',
                 data=data, return_type='dataframe', NA_action = "drop")
cluster = data[pd.notnull(data.avescores)].distname
psweights = data[pd.notnull(data.avescores)].psweight
mod = smf.WLS(y, X, weights = psweights)    # Describe model
res = mod.fit(cov_type = 'cluster',  cov_kwds={'groups': pd.DataFrame(cluster)})       # Fit model
print(res.summary())   # Summarize model

                            WLS Regression Results                            
Dep. Variable:              avescores   R-squared:                       0.061
Model:                            WLS   Adj. R-squared:                  0.060
Method:                 Least Squares   F-statistic:                     64.59
Date:                Thu, 14 Mar 2019   Prob (F-statistic):           2.91e-82
Time:                        21:33:18   Log-Likelihood:                -7285.1
No. Observations:                6302   AIC:                         1.459e+04
Df Residuals:                    6293   BIC:                         1.465e+04
Df Model:                           8                                         
Covariance Type:              cluster                                         
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept             0.3168      0.06

# Weights and pretest

In [68]:
y, X = dmatrices('avescores ~  treat + post + doi_year_centered + yearpost + treatyear + treatpost + treatpostyear + treatpostyear2 + scores15',
                 data=data, return_type='dataframe', NA_action = "drop")
cluster = data[pd.notnull(data.avescores)].distname
psweights = data[pd.notnull(data.avescores)].psweight
mod = smf.WLS(y, X, weights = psweights)    # Describe model
res = mod.fit(cov_type = 'cluster',  cov_kwds={'groups': pd.DataFrame(cluster)})       # Fit model
print(res.summary())   # Summarize model

                            WLS Regression Results                            
Dep. Variable:              avescores   R-squared:                       0.845
Model:                            WLS   Adj. R-squared:                  0.845
Method:                 Least Squares   F-statistic:                     733.8
Date:                Thu, 14 Mar 2019   Prob (F-statistic):               0.00
Time:                        21:33:44   Log-Likelihood:                -1609.4
No. Observations:                6302   AIC:                             3239.
Df Residuals:                    6292   BIC:                             3306.
Df Model:                           9                                         
Covariance Type:              cluster                                         
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept             0.1107      0.01